## Постановка задачи
Необходимо спрогнозировать, посетит ли клиент забронированное занятие, на основе истории посещаемости тренировок клиента. 
Цель - выявить закономерность и факторы,влияющие на неявку. 

### Подключение модулей

In [80]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%pip install seaborn
import seaborn as sns

### Загрузка данных

In [81]:
df = pd.read_csv("fitness_class_2212.csv")

In [82]:
df.head(10)

,booking_id,months_as_member,weight,days_before,day_of_week,time,category,attended
0,1,17,79.56,8,Wed,PM,Strength,0
1,2,10,79.01,2,Mon,AM,HIIT,0
2,3,16,74.53,14,Sun,AM,Strength,0
3,4,5,86.12,10,Fri,AM,Cycling,0
4,5,15,69.29,8,Thu,AM,HIIT,0
5,6,7,93.33,2,Mon,AM,Cycling,0
6,7,11,88.59,6,Wednesday,PM,HIIT,0
7,8,9,89.52,10,Fri,AM,HIIT,0
8,9,23,71.12,10,Fri.,AM,HIIT,1
9,10,7,81.25,10,Fri,AM,HIIT,0


### Описание данных

Признаки:
- booking_id - Уникальный идентификатор бронирования(ID бронирования)
- months_as_member - Сколько месяцев человек является клиентом фитнес-клуба
- weight - Вес клиента
- days_before - За сколько дней до занятия была сделана бронь
- day_of_week - День тренировки
- time - Запланированное время тренировки(день/вечер)
- category - Тип тренировки

Целевой (искомый) признак:

- attended - Факт посещения занятия

In [83]:
df.shape

(1500, 8)

Вывели общее количество строк и столбцов

In [84]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   booking_id        1500 non-null   int64  
 1   months_as_member  1500 non-null   int64  
 2   weight            1480 non-null   float64
 3   days_before       1500 non-null   object 
 4   day_of_week       1500 non-null   object 
 5   time              1500 non-null   object 
 6   category          1500 non-null   object 
 7   attended          1500 non-null   int64  
dtypes: float64(1), int64(3), object(4)
memory usage: 70.4+ KB


Данные содержат 1500 записей и 8 столбцов с различными характеристиками клиентов.
Есть 20 пропусков в столбце "weight"

In [85]:
df.describe()

,booking_id,months_as_member,weight,attended
count,1500.000000,1500.000000,1480.000000,1500.000000
mean,750.500000,15.628667,82.610378,0.302667
std,433.157015,12.926543,12.765859,0.459565
min,1.000000,1.000000,55.410000,0.000000
25%,375.750000,8.000000,73.490000,0.000000
50%,750.500000,12.000000,80.760000,0.000000
75%,1125.250000,19.000000,89.520000,1.000000
max,1500.000000,148.000000,170.520000,1.000000


Есть аномалии в весе и в продолжительности посещения клиенты/клиентов клуба.

In [86]:
df[df['months_as_member'] > 60]['months_as_member'].value_counts()

months_as_member
73     2
66     2
65     2
76     1
62     1
105    1
90     1
107    1
89     1
61     1
97     1
93     1
148    1
111    1
69     1
Name: count, dtype: int64

Сначала проанализируем клиентов, которые посещают клуб более 5 лет

In [87]:
df = df[df['months_as_member'] <= 60].copy()

Удалили аномалии в посещаемости. Остались только клиенты,которые посещают клуб не более 5 лет.

In [88]:
df[df['weight'] > 120]['weight'].value_counts()

weight
148.02    1
124.30    1
135.18    1
122.36    1
120.58    1
170.52    1
121.38    1
123.90    1
120.81    1
127.59    1
136.02    1
133.36    1
129.28    1
Name: count, dtype: int64

In [ ]:
Также проанализируем клиентов с аномальным весом

In [89]:
df = df[df['weight'] <= 140].copy()

Удалили аномалии в весе клиентов.Остались клиенты с весом меньше 140 кг

In [95]:
df.isna().sum()

booking_id          0
months_as_member    0
weight              0
days_before         0
day_of_week         0
time                0
category            0
attended            0
dtype: int64

Пропуски не найдены.

In [97]:
(df == "-").sum()

booking_id           0
months_as_member     0
weight               0
days_before          0
day_of_week          0
time                 0
category            13
attended             0
dtype: int64

Есть 13 пропусков в столбце категории.

In [100]:
df = df[df['category'] != "-"].copy()

In [101]:
(df == "-").sum()

booking_id          0
months_as_member    0
weight              0
days_before         0
day_of_week         0
time                0
category            0
attended            0
dtype: int64

In [ ]:
Удалили данные пропуски.

In [105]:
print(df.duplicated().sum())

0


Дубликатов не найдено.